def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas  y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [5]:
#IMPORTAR LIBRERIAS
import pandas as pd

In [6]:
#USAR SOLO LAS COLUMNAS QUE SIRVEN
df_counter=pd.read_csv("game_reviews2.csv", usecols=["user_id","posted","recommend"])

In [7]:
#DIVIDIR LAS COLUMNAS DE FECHA PARA PODER LIMPIAR LOS DATOS
df1=df_counter["posted"].str.split(expand=True)

In [8]:
#ESCOGER LAS COLUMNAS UTILES
df1=df1.iloc[:,1:4]

In [9]:
#UNIR COLUMNAS QUE FUERON DIVIDIDAS
df_counter=pd.concat([df_counter,df1], axis='columns').drop(columns=["posted"])

In [10]:
#RENOMBRAS LAS COLUMNAS
df_counter.rename(columns={1:"Month",2:"Day",3:"Year"}, inplace=True)

In [11]:
df_counter

,user_id,recommend,Month,Day,Year
0,76561197970982479,True,November,"5,",2011.
1,76561197970982479,True,July,"15,",2011.
2,76561197970982479,True,April,"21,",2011.
3,js41637,True,June,"24,",2014.
4,js41637,True,September,"8,",2013.
...,...,...,...,...,...
59328,76561198312638244,True,July,10.,None
59329,76561198312638244,True,July,8.,None
59330,LydiaMorley,True,July,3.,None
59331,LydiaMorley,True,July,20.,None


In [12]:
#LIMPIAR LA PUNTUACION EN LAS COLUMNAS PARA LUEGO PODER UNIRLAS
import string
df_counter["Day"] = df_counter["Day"].str.replace('[{}]'.format(string.punctuation), '',regex=True)
df_counter["Year"] = df_counter["Year"].str.replace('[{}]'.format(string.punctuation), '',regex=True)

In [13]:
#REEMPLAZAR LOS AÑOS QUE NO ESTAN CON LA MODA
df_counter['Year'].fillna(df_counter['Year'].mode()[0], inplace=True)

In [14]:
#REEMPLAZAR LA COLUMNA DE MESES CON LOS MESES NUMERICOS
df_counter["Month"].replace({"January":"01","February":"02","March":"03","April":"04","May":"05",
     "June":"06","July":"07","August":"08","September":"09","October":"10","November":"11","December":"12"},inplace=True)

In [15]:
#CREAR UNA COLUMNA DE FECHA DONDE ESTAN JUNTAS LAS FILAS DE DIA, MES, AÑO
df_counter["Fecha"]= df_counter["Year"]+"-"+df_counter["Month"]+"-"+df_counter["Day"]

In [16]:
#CONVERTIR LA COLUMNA DE FECHA EN TIPO STRING
df_counter["Fecha"]=df_counter["Fecha"].astype(str)

In [17]:
#HACER LA FUNCION QUE CONVIERTE LA FECHA EN TIPO DATETIME
def time(df):
    for index,item in enumerate(df["Fecha"]):
        try:
            datetime.strptime(item, "%Y-%m-%d").date()
        
        except:
            print(index)
            print(item)

In [18]:
#REEMPLAZAR LOS VALORES DIFERENTES COMO LA DEL 29 DE FEBRERO
df_counter["Fecha"].replace({"2014-02-29":"2014-02-28"}, inplace=True)

In [19]:
#ENCONTRAR LA MODA PARA REEMPLAZARLO EN LA FECHA
df_counter["Fecha"].mode()

0    2014-06-21
Name: Fecha, dtype: object

In [20]:
#REEMPLAZAR LA MODA EN LAS FILAS DONDE NO HAY VALORES
df_counter["Fecha"].replace({"nan":"2014-06-21"}, inplace=True)

In [ ]:
#APLICAR LA FUNCION A LA DF_COUNTER
time(df_counter)

In [22]:
#ELIMINAR LAS COLUMNAS QUE NO SIRVEN 
df_counter.drop(columns=["Month","Day","Year"],inplace=True)

In [139]:
df_counter.to_csv("df_counter_func_2.csv",index=False)

In [24]:
df_counter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   recommend  59305 non-null  object
 2   Fecha      59333 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [30]:
#CONTAR LOS REVIEWS POR FECHAS
(df_counter[df_counter["Fecha"]>"2011-11-5"]["recommend"]).nunique()

2

In [25]:
df_counter["user_id"].nunique()

25485

In [29]:
#CONTAR LOS USER POR FECHAS
df_counter[(df_counter["Fecha"]>"2011-11-5")& (df_counter["Fecha"]<"2012-01-30")]["user_id"].nunique()

170

In [35]:
#CONTAR LAS REVIEWS POR FECHAS
recommend=df_counter[(df_counter["Fecha"]>"2011-11-5")& (df_counter["Fecha"]<"2014-07-8")]["recommend"]

In [38]:
recommend.value_counts()[True]

27470

In [32]:
#ENCONTRAR EL PORCENTAJE DE RECOMENDACIONES 
porce_recom=(recommend.value_counts()[True])/len(recommend)

In [43]:
#FUNCION DE RECOMENDACION
def countreviews( fecha1,fecha2 : str ):
    df_counter=pd.read_csv("df_counter_func_2.csv")
    cantidad_usu_rese=df_counter[(df_counter["Fecha"]>fecha1)& (df_counter["Fecha"]<fecha2)]["user_id"].nunique()
    recommend=df_counter[(df_counter["Fecha"]>fecha1)& (df_counter["Fecha"]<fecha2)]["recommend"]
    porce_recom=round((recommend.value_counts()[True])/len(recommend)*100,2)
    response={"Cantidad de usuarios que reseñaron":cantidad_usu_rese,"Porcentaje de recomendacion":porce_recom}

    return response

In [44]:
countreviews("2014-01-01","2014-12-31")

{'Cantidad de usuarios que reseñaron': 17628,
 'Porcentaje de recomendacion': 88.01}